# pandas-常用操作函数

In [2]:
import time
import random

from datetime import datetime

import pandas as pd
import numpy as np

# 1 数据操作 - Data manipulations

## 1.1 melt / wide_to_long

#### 案例

In [3]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [4]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [5]:
pd.melt(df, id_vars=['A'], value_vars=['B'],
        var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


In [6]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


#### 案例2

In [7]:
df.columns = [list('ABC'), list('DEF')]
df

,A,B,C
,D,E,F
0,a,1,2
1,b,3,4
2,c,5,6


In [8]:
pd.melt(df, col_level=0, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [9]:
pd.melt(df, id_vars=[('A', 'D')], value_vars=[('B', 'E')])

,"(A, D)",variable_0,variable_1,value
0,a,B,E,1
1,b,B,E,3
2,c,B,E,5


## 1.2 concat

#### 功能：

在一个特定的轴上，沿着另一个坐标轴，将 pandas 对象连接起来。

`pandas.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)[source] `

#### 参数说明
1. objs
2. axis=0
3. join='outer'
4. join_axes=None
5. ignore_index=False
6. keys=None
7. levels=None
8. names=None
9. verify_integrity=False
10. copy=True

#### 返回值  
obj, type of objs



#### 案例

In [10]:
# 造 df 的函数
def create_df(cols,indexs):
    content_dic = {}
    for col in cols:
        content_arr = []
        for index in indexs:
            content = str(col)+str(index)
            content_arr.append(content)
        content_dic[col] = content_arr
    return pd.DataFrame(content_dic,index=indexs)

In [11]:
cols = list('ABCDE')
index = list('12345')
df_a = create_df(cols,index)
df_a

,A,B,C,D,E
1,A1,B1,C1,D1,E1
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4
5,A5,B5,C5,D5,E5


In [12]:
cols = list('BCDEG')
index = list('23456')
df_b = create_df(cols,index)
df_b

,B,C,D,E,G
2,B2,C2,D2,E2,G2
3,B3,C3,D3,E3,G3
4,B4,C4,D4,E4,G4
5,B5,C5,D5,E5,G5
6,B6,C6,D6,E6,G6


In [13]:
## 1.0 简单的合并 (默认参数：axis=0, join='outer', ignore_index=False, verify_integrity=False, copy=True)
df_c = pd.concat([df_a,df_b])
df_c

,A,B,C,D,E,G
1,A1,B1,C1,D1,E1,NaN
2,A2,B2,C2,D2,E2,NaN
3,A3,B3,C3,D3,E3,NaN
4,A4,B4,C4,D4,E4,NaN
5,A5,B5,C5,D5,E5,NaN
2,NaN,B2,C2,D2,E2,G2
3,NaN,B3,C3,D3,E3,G3
4,NaN,B4,C4,D4,E4,G4
5,NaN,B5,C5,D5,E5,G5
6,NaN,B6,C6,D6,E6,G6


In [14]:
## 1.1  简单的合并 (join='inner', 取交集；join='inner', 取并集)
df_c = pd.concat([df_a,df_b],axis=1,join='inner')
df_c

,A,B,C,D,E,B,C,D,E,G
2,A2,B2,C2,D2,E2,B2,C2,D2,E2,G2
3,A3,B3,C3,D3,E3,B3,C3,D3,E3,G3
4,A4,B4,C4,D4,E4,B4,C4,D4,E4,G4
5,A5,B5,C5,D5,E5,B5,C5,D5,E5,G5


In [15]:
## 1.2  简单的合并 (ignore_index=True, 重新整理 index / cols)
df_c = pd.concat([df_a,df_b],axis=1,join='inner',ignore_index=True) 
df_c

,0,1,2,3,4,5,6,7,8,9
2,A2,B2,C2,D2,E2,B2,C2,D2,E2,G2
3,A3,B3,C3,D3,E3,B3,C3,D3,E3,G3
4,A4,B4,C4,D4,E4,B4,C4,D4,E4,G4
5,A5,B5,C5,D5,E5,B5,C5,D5,E5,G5


In [19]:
## 1.3  简单的合并，避免合并重复索引 (verify_integrity=False, 验证合并方向的 index 或 cols 是否有重复，有重复会报错)
try:
    df_c = pd.concat([df_a,df_b],axis=0,join='inner', verify_integrity=True)
except Exception as e:
    print(e)

Indexes have overlapping values: ['2', '3', '4', '5']


In [20]:
## 1.3  简单的合并 (copy=True, ?? 没有发现区别)
df_c = pd.concat([df_a,df_b],copy=False)
df_c

,A,B,C,D,E,G
1,A1,B1,C1,D1,E1,NaN
2,A2,B2,C2,D2,E2,NaN
3,A3,B3,C3,D3,E3,NaN
4,A4,B4,C4,D4,E4,NaN
5,A5,B5,C5,D5,E5,NaN
2,NaN,B2,C2,D2,E2,G2
3,NaN,B3,C3,D3,E3,G3
4,NaN,B4,C4,D4,E4,G4
5,NaN,B5,C5,D5,E5,G5
6,NaN,B6,C6,D6,E6,G6


In [21]:
## 1.4  简单的合并 (copy=True, ?? 没有发现区别)
df_c = pd.concat([df_a,df_b],copy=False)
df_c

,A,B,C,D,E,G
1,A1,B1,C1,D1,E1,NaN
2,A2,B2,C2,D2,E2,NaN
3,A3,B3,C3,D3,E3,NaN
4,A4,B4,C4,D4,E4,NaN
5,A5,B5,C5,D5,E5,NaN
2,NaN,B2,C2,D2,E2,G2
3,NaN,B3,C3,D3,E3,G3
4,NaN,B4,C4,D4,E4,G4
5,NaN,B5,C5,D5,E5,G5
6,NaN,B6,C6,D6,E6,G6


In [22]:
## 2.0 可选的合并选项(join_axes=None, keys=None, levels=None, names=None)
## 2.1.1 选择一个df的列索引作为依据，会删除其他的选项 (join_axes=[df_a.columns],axis=0 配合使用)
df_c = pd.concat([df_a,df_b],join_axes=[df_a.columns],axis=0)
df_c

,A,B,C,D,E
1,A1,B1,C1,D1,E1
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4
5,A5,B5,C5,D5,E5
2,NaN,B2,C2,D2,E2
3,NaN,B3,C3,D3,E3
4,NaN,B4,C4,D4,E4
5,NaN,B5,C5,D5,E5
6,NaN,B6,C6,D6,E6


In [23]:
## 2.1.2 选择一个df的行索引作为依据，会删除其他的选项 (join_axes=[df_a.index],axis=1 配合使用)
df_c = pd.concat([df_a,df_b],join_axes=[df_a.index],axis=1)
df_c

,A,B,C,D,E,B,C,D,E,G
1,A1,B1,C1,D1,E1,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,E2,B2,C2,D2,E2,G2
3,A3,B3,C3,D3,E3,B3,C3,D3,E3,G3
4,A4,B4,C4,D4,E4,B4,C4,D4,E4,G4
5,A5,B5,C5,D5,E5,B5,C5,D5,E5,G5


In [24]:
## 2.2 对每个合并的 df 添加一级索引，作为标签 (keys=['x','y'])
## 适用于：合并相同索引的数据，区分不同索引选项
## levels=None, names=None 和 key 有关，不常用
df_c = pd.concat([df_a,df_b],keys=['x','y'])
df_c

A   B   C   D   E    G
x 1   A1  B1  C1  D1  E1  NaN
  2   A2  B2  C2  D2  E2  NaN
  3   A3  B3  C3  D3  E3  NaN
  4   A4  B4  C4  D4  E4  NaN
  5   A5  B5  C5  D5  E5  NaN
y 2  NaN  B2  C2  D2  E2   G2
  3  NaN  B3  C3  D3  E3   G3
  4  NaN  B4  C4  D4  E4   G4
  5  NaN  B5  C5  D5  E5   G5
  6  NaN  B6  C6  D6  E6   G6

## 1.3 merge / merge_ordered / merge_asof

## 1.3.1 merge

#### merge与concat的区别在于:  
1. merge需要依据某一共同的列或者列索引来进行合并
2. 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。
3. 注意每一列元素的顺序不要求一致

`pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, 
          left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), 
          copy=True, indicator=False, validate=None)`
#### 参数说明：
1. left, right: DataFrame
2. how: {'left', 'right', 'outer', 'inner'}, default 'inner'
3. on, left_on, right_on: label or list
    - on: 两边 DF 都拥有的key；如果不指定，默认合并列的交集；当有多个列名相同时，需要指定 on 的值；
    - left_on, right_on：如果两边的 DF 列名不一致，需要指定。
4. left_index=False, right_index=False  
    - left_index: 将左侧的行索引用作其连接键
    - right_index: 将右侧的行索引用作其连接键 
5. sort=False  
    - 根据连接键对合并后的数据进行排序，默认为False。
    - 有时在处理大数据集时，禁用该选项可获得更好的性能。
6. suffixes=('_x', '_y')
    - 当有多个列名相同时，需要指定非 on 的列的后缀加以区分；
7. indicator=False
    
8. validate: string, {'one_to_one', 'one_to_many', 'many_to_one', 'many_to_many'}
    验证合并是否为指定类型：
    - “one_to_one”或“1:1”:检查合并键是否在左右数据集中都是唯一的。
    - “one_to_many”或“1:m”:检查合并键是否在左数据集中是唯一的。 
    - “许多_to_one”或“m:1”:检查合并键是否在正确的数据集中是唯一的。
    - “many_to_many”或“m:m”:允许，但不会导致检查。

#### 返回值：DataFrame


#### 案例

## 1.3.2 merge_ordered

## 1.3.3 merge_asof

## 1.4 cut / qcut

### 1.4.1 cut

#### 功能：  
将数据进行离散化

####  参数说明  
`pandas.cut(x,bins,right=True,labels=None,retbins=False,precision=3,include_lowest=False)`  

1. x: 进行划分的一维数组  
2. bins: 
    - 整数，将x划分为多少个等间距的区间；
    - 序列，将x划分在指定的序列中，若不在该序列中，则是NaN。
3. right: 是否包含右端点  
4. labels: 是否用标记来代替返回的bins  
5. precision: 精度
6. include_lowest: 是否包含左端点
7. retbins:
    - retbins = False: 则返回x中每个值对应的bin的列表；
    - retbins = True: 则返回x中每个值对应的bin的列表和对应的bins。

####  案例

In [25]:
# bins = int, 划分为等区间间距的
data_r = pd.cut(np.array([0.2,1.4,2.5,6.2,9.7,2.1]),3,retbins=True)   
data_r

([(0.19, 3.367], (0.19, 3.367], (0.19, 3.367], (3.367, 6.533], (6.533, 9.7], (0.19, 3.367]]
 Categories (3, interval[float64]): [(0.19, 3.367] < (3.367, 6.533] < (6.533, 9.7]],
 array([0.1905    , 3.36666667, 6.53333333, 9.7       ]))

In [26]:
data_r[0],data_r[1]  # 通过切片取结果值

([(0.19, 3.367], (0.19, 3.367], (0.19, 3.367], (3.367, 6.533], (6.533, 9.7], (0.19, 3.367]]
 Categories (3, interval[float64]): [(0.19, 3.367] < (3.367, 6.533] < (6.533, 9.7]],
 array([0.1905    , 3.36666667, 6.53333333, 9.7       ]))

In [27]:
# bins = int, 划分为等区间间距的
pd.cut(np.array([0.2,1.4,2.5,6.2,9.7,2.1]),3,retbins=False)

[(0.19, 3.367], (0.19, 3.367], (0.19, 3.367], (3.367, 6.533], (6.533, 9.7], (0.19, 3.367]]
Categories (3, interval[float64]): [(0.19, 3.367] < (3.367, 6.533] < (6.533, 9.7]]

In [28]:
# bins = 序列, 划分为指定序列中，不在序列中，则返回nan
pd.cut(np.array([0.2,1.4,2.5,6.2,9.7,2.1]),[1,2,3],retbins=True)

([NaN, (1, 2], (2, 3], NaN, NaN, (2, 3]]
 Categories (2, interval[int64]): [(1, 2] < (2, 3]], array([1, 2, 3]))

In [29]:
# 是否用标记来代替返回的bins
pd.cut([1,2,3,4],4,labels=['one','two','three','four'])

[one, two, three, four]
Categories (4, object): [one < two < three < four]

## 1.4.2 qcut

#### 功能
将数据进行离散化。

#### 参数说明
`pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates=’raise’)`

1. x: 1d数组 或 series  
2. q: int or 分位数数组  
3. labels: array or boolean, default None 
标签。必须分位区间数相同长度；否则，只返回整数部分分位区间。
4. retbins: bool, optional  
    - retbins = False: 则返回x中每个值对应的bin的列表；
    - retbins = True: 则返回x中每个值对应的bin的列表和对应的bins。
5. percision: int, optional  
精度
6. duplicates: {default ‘raise’, ‘drop’}, optional  
如果 q 边界不是无重复的，会报 `ValueError` 或者 删除非重复项。 

#### 案例

In [30]:
pd.qcut(range(5), 4)

[(-0.001, 1.0], (-0.001, 1.0], (1.0, 2.0], (2.0, 3.0], (3.0, 4.0]]
Categories (4, interval[float64]): [(-0.001, 1.0] < (1.0, 2.0] < (2.0, 3.0] < (3.0, 4.0]]

In [31]:
pd.qcut(range(5), 3, labels=["good", "medium", "bad"])

[good, good, medium, bad, bad]
Categories (3, object): [good < medium < bad]

In [32]:
pd.qcut(range(5), 4, labels=False)

array([0, 0, 1, 2, 3], dtype=int64)

# 1.5 pivot / pivot_table / crosstab

## 1.5.1 pivot

## 1.5.2 pivot_table

## 1.5.3 crosstab

## 1.6 get_dummies

## 1.7 factorize

## 1.8 unique

# 2 空值相关函数

# 3 转换函数

# 4 时间序列

# 5 间隔函数 - dealing with intervals

# 6 评估

# 7 测试